# Preprocesamiento de datos de texto

In [1]:
import datasets  # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import es_core_news_sm  # Modelo Spacy de procesamiento de lenguaje natural en español
import spacy  # Biblioteca de procesamiento de lenguaje natural
import pandas as pd  # Biblioteca de manejo de conjuntos de datos
import re  # Módulo de expresiones regulares
import tokenizers  # Biblioteca de tokenización de texto
import nltk  # Biblioteca de procesamiento de lenguaje natural
from pathlib import Path  # Biblioteca para manejo de paths relativos

## 🤗 Datasets

🤗 (HuggingFace) Datasets es una biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural que se destaca por la simplicidad de sus métodos y el gran repositorio 🤗 Hub que contiene muchos conjuntos de datos libres para descargar sólo con una linea de Python.

En nuestro curso trabajaremos con `spanish_diagnostics`, un conjunto de datos de nuestro grupo investigación PLN@CMM que contiene textos de sospechas diagnósticas de la lista de espera chilena y está etiquetado con el destino de la interconsulta; este destino puede ser `dental` o `no_dental`.

In [2]:
# Con esta linea descargamos el conjunto de datos completo
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics')

In [3]:
def normalize(text, remove_tildes=True):
    """Normaliza una cadena de texto convirtiéndo todo a minúsculas, quitando los caracteres no alfabéticos y los tildes"""
    text = text.lower()  # Llevamos todo a minúscula
    # Reemplazamos los caracteres no alfabéticos por un espacio
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text)
    if remove_tildes:
        text = re.sub('á', 'a', text)  # Reemplazamos los tildes
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

In [4]:
spanish_diagnostics_normalized = spanish_diagnostics["train"].map(
    lambda x: {  # Utilizamos una función anónima que devuelve un diccionario
        # Esta es una nueva característica que agregaremos a nuestro conjunto de datos.
        "normalized_text": normalize(x["text"])
    })

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

In [5]:
spanish_diagnostics_normalized

Dataset({
    features: ['text', 'label', 'normalized_text'],
    num_rows: 70000
})

Ahora nuestro conjunto de datos cuenta con una nueva característica `normalized_text`.

In [6]:
spanish_diagnostics_normalized[0]

{'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'label': 1,
 'normalized_text': '  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'}

## Tokenización

La tokenización es el proceso de demarcación de secciones de una cadena de caracteres. Estas secciones podrían ser oraciones, palabras o subpalabras.

El método más simple para tokenizar una cadena de caracteres en nuestro lenguaje es la separación por espacios. Aplicamos una separación por espacios mediante el método `str.split()` sobre nuestro conjunto de datos normalizado.

In [7]:
spanish_diagnostics_normalized[0]["normalized_text"]

'  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'

In [8]:
spanish_diagnostics_normalized[0]["normalized_text"].split()

['anomalias',
 'dentofaciales',
 'incluso',
 'la',
 'maloclusion',
 'discrepancia',
 'dentomaxilar']

Si bien el método de separación por espacios funciona bien en nuestro conjunto de datos normalizado, también quisiéramos tokenizar nuestro texto sin normalizar.

In [9]:
spanish_diagnostics_normalized[0]["text"]

'- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR'

In [10]:
spanish_diagnostics_normalized[0]["text"].split()

['-',
 'ANOMALÍAS',
 'DENTOFACIALES',
 '(INCLUSO',
 'LA',
 'MALOCLUSIÓN)',
 'DISCREPANCIA',
 'DENTOMAXILAR']

Al aplicar el mismo métodos podemos observar que no funciona totalmente bien debido a la presencia de caracteres no alfabéticos. Para solucionar esto, existen métodos basados en una serie de reglas para solucionar estos problemas. Utilizaremos la implementación de un tokenizador basado en reglas de la biblioteca de procesamiento de lenguaje natural Spacy.

In [11]:
spacy_tokenizer = es_core_news_sm.load().tokenizer

In [12]:
list(spacy_tokenizer(spanish_diagnostics_normalized[0]["text"]))

[-,
 ANOMALÍAS,
 DENTOFACIALES,
 (,
 INCLUSO,
 LA,
 MALOCLUSIÓN,
 ),
 
 
 
  ,
 DISCREPANCIA,
 DENTOMAXILAR]

Al utilizar el tokenizador basado en reglas, podemos tener resultados mucho mejores que los anteriores.

### 🤗 Tokenizers

🤗 también cuenta con una biblioteca llamada Tokenizers, con la cual podemos construir nuestro tokenizador basado en nuestro conjunto de datos.

Instanciamos el tokenizador con un modelo WordPiece, el cual parte construyendo un vocabulario que incluye todas los caracteres presentes en el conjunto de datos y posteriormente comienza a mezclar caracteres hasta encontrar conjuntos de caracteres que tienen más probabilidad de aparecer juntos que separados.

In [13]:
tokenizer = tokenizers.Tokenizer(tokenizers.models.WordPiece())

Esta biblioteca nos permite añadir pasos de normalización directamente. Replicamos lo mismo que hacemos con nuestra función `normalizer()`.

In [14]:
normalizer = tokenizers.normalizers.Sequence([
    tokenizers.normalizers.Lowercase(),  # Llevamos todo a minúscula
    # Separamos cada caracter según los elementos que lo componen: á -> (a, ´)
    tokenizers.normalizers.NFD(),
    tokenizers.normalizers.StripAccents(),  # Eliminamos todos los acentos
    # Reemplazamos todos los caracteres no alfabéticos
    tokenizers.normalizers.Replace(tokenizers.Regex(r"[^a-z ]"), " ")
])

In [15]:
normalizer.normalize_str(spanish_diagnostics_normalized[0]["text"])

'  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'

Añadimos el normalizador al tokenizador

In [16]:
tokenizer.normalizer = normalizer

Pre tokenizamos nuestro conjunto de datos mediante espacio para delimitar el tamaño que puede tener cada token.

In [17]:
tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()

Instanciamos el entrenador que entrenará nuestro tokenizador.

In [18]:
trainer = tokenizers.trainers.WordPieceTrainer()

Entrenamos el tokenizador sobre nuestro conjunto de datos.

In [19]:
tokenizer.train_from_iterator(spanish_diagnostics_normalized["text"], trainer)

Mediante el método `Tokenizer.encode()` obtenemos la representación tokenizada de nuesto texto. Esta representación contiene varios atributos, donde los más interesantes son:

- `ids`: Contiene nuestro texto representado a través de una lista que contiene los identificadores de cada token.
- `tokens`: Contiene nuestro texto representado a través de una lista que contiene el texto de cada token.

In [20]:
spanish_diagnostics_normalized[0]["text"]

'- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR'

In [21]:
tokenized_output = tokenizer.encode(spanish_diagnostics_normalized[0]["text"])

In [22]:
# Para ver todo el vocabulario del tonekizador consultar la siguiente funcion
tokenizer.get_vocab()

{'hirat': 22849,
 'fun': 8799,
 '##ilit': 716,
 'espind': 18084,
 'crepitacion': 5609,
 'pulpalgia': 18213,
 '##entomaxilar': 17602,
 '##nan': 24155,
 '##esiona': 18491,
 'contaminado': 21448,
 '##fue': 17492,
 'correcta': 5250,
 '##rx': 17303,
 'dl': 1440,
 'alterado': 3302,
 'saber': 9269,
 'superir': 11653,
 'pulper': 27631,
 'somet': 7468,
 'biopsiado': 13899,
 'obbs': 27022,
 'insulinorrequiriente': 13496,
 'subclases': 21798,
 'consecutivos': 16078,
 '##versdion': 28854,
 'vvi': 14244,
 '##ilar': 229,
 '##yl': 14387,
 'emp': 4504,
 'marzo': 4961,
 'inoclusion': 7572,
 '##jeto': 15753,
 'esternosis': 16523,
 'hrt': 4307,
 'capucho': 21387,
 'despulpado': 8548,
 'aricular': 28515,
 'sociable': 23382,
 'defecit': 21162,
 '##isfuncion': 25252,
 'concordancia': 20153,
 'neuritis': 7351,
 'incapacitante': 13930,
 'bor': 11030,
 'trasera': 18548,
 'dfecto': 21502,
 'semis': 10376,
 '##iscopatia': 13488,
 'infrapat': 20501,
 'faltan': 11556,
 'duras': 15644,
 'gastos': 19102,
 '##eb': 68

In [23]:
tokenized_output.ids

[383, 618, 635, 119, 466, 1765, 970]

In [24]:
tokenized_output.tokens

['anomalias',
 'dentofaciales',
 'incluso',
 'la',
 'maloclusion',
 'discrepancia',
 'dentomaxilar']

Tal como lo hicimos anteriormente podemos aplicar paralelamente nuestro tokenizador sobre el conjunto de datos mediante el método `Dataset.map()`

In [25]:
spanish_diagnostics_normalized_tokenized = spanish_diagnostics_normalized.map(
    lambda x: {"tokenized_text": tokenizer.encode(x["text"]).tokens})

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Nuestro conjunto de datos ahora contiene el texto tokenizado en la característica `tokenized_text`.

In [26]:
spanish_diagnostics_normalized_tokenized[0]

{'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'label': 1,
 'normalized_text': '  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar',
 'tokenized_text': ['anomalias',
  'dentofaciales',
  'incluso',
  'la',
  'maloclusion',
  'discrepancia',
  'dentomaxilar']}

## Stemming y Lematización

Con el fin de disminuir la cantidad de características de las representaciones de texto existen métodos que reducen el tamaño de vocabulario al eliminar inflexiones que puedan tener las palabras. Estos métodos son:

- Lematización: Este método lleva una palabra en su forma flexionada a su forma base, por ejemplo *tratada* -> *tratar*
- Stemming: Este método trunca las palabras de entrada mediante un algoritmo predefinido para encontrar la raíz de la misma, por ejemplo *tratada* -> *trat*

El proceso de lematización lo haremos a través de la biblioteca Spacy y el proceso de stemming a través de la biblioteca NLTK utilizando el algoritmo Snowball.

Instanciamos el analizador de Spacy

In [27]:
nlp = es_core_news_sm.load()

Definimos como tokenizador el que entrenamos anteriormente.

In [28]:
def custom_tokenizer(text):
    tokens = tokenizer.encode(text).tokens
    return spacy.tokens.Doc(nlp.vocab, tokens)

In [29]:
nlp.tokenizer = custom_tokenizer

Instanciamos el Stemmer

In [30]:
stemmer = nltk.stem.SnowballStemmer("spanish")

Podemos verificar cómo funcionan estos métodos sobre un texto de prueba de nuestro conjunto de datos.

In [31]:
spanish_diagnostics_normalized_tokenized[5]["text"]

'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria'

In [32]:
for t in nlp(spanish_diagnostics_normalized_tokenized[5]["text"]):
    print(
        f"Token: {t.text}\nLema: {t.lemma_}\nStem: {stemmer.stem(t.text)}\n---")

Token: pieza
Lema: pieza
Stem: piez
---
Token: n
Lema: n
Stem: n
---
Token: tratada
Lema: tratado
Stem: trat
---
Token: endodonticamente
Lema: endodonticamente
Stem: endodont
---
Token: restaurada
Lema: restaurado
Stem: restaur
---
Token: con
Lema: con
Stem: con
---
Token: ionomero
Lema: ionomero
Stem: ionomer
---
Token: y
Lema: y
Stem: y
---
Token: resina
Lema: resina
Stem: resin
---
Token: compuesta
Lema: compuesto
Stem: compuest
---
Token: necesita
Lema: necesitar
Stem: necesit
---
Token: protesis
Lema: protesis
Stem: protesis
---
Token: fija
Lema: fijo
Stem: fij
---
Token: por
Lema: por
Stem: por
---
Token: gran
Lema: gran
Stem: gran
---
Token: pnrdida
Lema: pnrdida
Stem: pnrdid
---
Token: coronaria
Lema: coronario
Stem: coronari
---
